# Intro 
In this notebook the top 10 live trends from Twitter are used and stored in a Google spreadsheet.


# Imports & Installs 

In [1]:
#!pip install tweepy
#!pip install -U gspread

In [ ]:
import numpy as np 
import pandas as pd
import tweepy
import gspread
from gspread_dataframe import set_with_dataframe
from datetime import datetime

# Inputs

In [ ]:
# Twitter Developer keys here
consumer_key = 'yyy'
consumer_key_secret = 'xxx'
access_token = 'zzz'
access_token_secret = 'qqq'

# Location code (here for USA)
location_code = 23424977 

# GSPREAD Details 
gs_read = "csv input here" # Format: https://docs.google.com/spreadsheets/d/yyyyyy-xxxxxxxxxxxxxx/export?gid=0&format=csv

gc = gspread.service_account(filename='appname-yyyy-xxxxxxxx.json') # Filename of the json file with the credentials from google api
sh = gc.open_by_key('yyyyyy-xxxxxxxxxxxxxx') # Spreadsheet key -> from the url above (yyyyyy-xxxxxxxxxxxxxx)
worksheet = sh.get_worksheet(0) # 0 = first sheet, 1 = second sheet etc. 

# Workflow


In [ ]:
# Twitter API setup 
auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# Get the Trends (limit to 10)
data = api.trends_place(location_code)
trends = data[0]["trends"]
trend_names = '\n'.join(trend['name'] for trend in trends[0:10])

# Get current time to save it in the dataframe/csv
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S") # Format: dd/mm/YY H:M:S

df = pd.DataFrame({"trends": [trend['name'] for trend in trends], "tweet_volume": [trend['tweet_volume'] for trend in trends], "date_time": dt_string})
df_sort = df.sort_values(by="tweet_volume", ascending=False, ignore_index=True)[:10] # Get the top 10 trends 

df_dl = pd.read_csv(gs_read) # load the actual csv to get the shape later on 

# Append the data to the spreadsheet
r_count = df_dl.shape[0] + 2 # to append to the last row - prevents overwriting 
set_with_dataframe(worksheet, df_sort, row=r_count, include_column_header=False) # Save the dataframe in the google spreadsheet